In [1]:
# note that this custom dataset is not prepared on the top of geometric Dataset(pytorch's inbuilt)
import os
import torch
import glob
import numpy as np
import math
import pandas as pd
from os import listdir
from os.path import isfile, join


c:\Users\laura\anaconda3\envs\warsztaty_badawcze\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # In PyG 1.*
# data = torch.load("./PPI_GNN-main/Human_features/processed/10GS.pt")
# torch.save(data.to_dict(), "./PPI_GNN-main/Human_features/processed_new/10GS.pt")

# # # In PyG 2.*
# # data_dict = torch.load('data_dict.pt')
# # data = Data.from_dict(data_dict)
# # torch.save(data, 'data.pt')

In [3]:
# for x in listdir("./PPI_GNN-main/Human_features/processed/"):
#     dir = os.path.join("./PPI_GNN-main/Human_features/processed/", x)
#     dir2 = os.path.join("./PPI_GNN-main/Human_features/processed_new/", x)
#     print(x)

#     # data = torch.load(dir)
#     # torch.save(data.to_dict(), dir2)

#     data_dict = torch.load(dir)
#     data = pd.DataFrame.from_dict(data_dict)
#     torch.save(data, dir2)

In [10]:

processed_dir = "./PPI_GNN-main/Human_features/processed/"
npy_file = "./PPI_GNN-main/Human_features/npy_file_new(human_dataset).npy"
npy_ar = np.load(npy_file)
print("SHAPE:", npy_ar.shape)
print(npy_ar)

from torch.utils.data import Dataset as Dataset_n
from torch_geometric.data import DataLoader as DataLoader_n

class LabelledDataset(Dataset_n):
    def __init__(self, npy_file, processed_dir):
      self.npy_ar = np.load(npy_file)
      self.processed_dir = processed_dir
      self.protein_1 = self.npy_ar[:,2]
      self.protein_2 = self.npy_ar[:,5]
      self.label = self.npy_ar[:,6].astype(float)
      self.n_samples = self.npy_ar.shape[0]

    def __len__(self):
      return(self.n_samples)

    def __getitem__(self, index):
      prot_1 = os.path.join(self.processed_dir, self.protein_1[index]+".pt")
      prot_2 = os.path.join(self.processed_dir, self.protein_2[index]+".pt")
      prot_1 = torch.load(glob.glob(prot_1)[0])
      prot_2 = torch.load(glob.glob(prot_2)[0])
      return prot_1, prot_2, torch.tensor(self.label[index])



dataset = LabelledDataset(npy_file = npy_file ,processed_dir=processed_dir)

final_pairs =  np.load(npy_file)
size = final_pairs.shape[0]
print("\nSize:", size)
seed = 42
torch.manual_seed(seed)
trainset, testset = torch.utils.data.random_split(dataset, [math.floor(0.8 * size), size - math.floor(0.8 * size) ])
trainloader = DataLoader_n(dataset= trainset, batch_size= 4, num_workers = 0)
testloader = DataLoader_n(dataset= testset, batch_size= 4, num_workers = 0)

print("\nLength:")
print(len(trainloader))
print(len(testloader))

SHAPE: (22217, 7)
[['NP_663777' 'Q13114' '1FLK' ... 'P26842' '5TL5' '1']
 ['NP_003630' 'Q9Y6K9' '2JVX' ... 'Q14790' '1F9E' '1']
 ['NP_003001' 'P45985' '3ALN' ... 'Q12852' '5CEN' '1']
 ...
 ['NP_000779' 'P27707' '1P5Z' ... 'P00338' '1I10' '0']
 ['NP_008828' 'P31274' '2LP0' ... 'Q9H257' '6E25' '0']
 ['NP_001740' 'P04632' '1KFU' ... 'Q15052' '1UJY' '1']]


TypeError: Expected 'Iterator' as the return annotation for `__iter__` of SMILESParser, but found typing.Any

In [38]:
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/5)


In [39]:
print("total_samples:", total_samples, "n_iterations:", n_iterations)

total_samples: 22217 n_iterations: 4444


In [40]:
# Building model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, global_max_pool as gmp, global_add_pool as gap,global_mean_pool as gep,global_sort_pool
from torch_geometric.utils import dropout_adj
from torch.optim.lr_scheduler import MultiStepLR



class GCNN(nn.Module):
    def __init__(self, n_output=1, num_features_pro= 1024, output_dim=128, dropout=0.2):
        super(GCNN, self).__init__()

        print('GCNN Loaded')

        # for protein 1
        self.n_output = n_output
        self.pro1_conv1 = GCNConv(num_features_pro, num_features_pro)
        self.pro1_fc1 = nn.Linear(num_features_pro, output_dim)

        # for protein 2
        self.pro2_conv1 = GCNConv(num_features_pro, num_features_pro)
        self.pro2_fc1 = nn.Linear(num_features_pro, output_dim)

        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

        # combined layers
        self.fc1 = nn.Linear(2 * output_dim, 256)
        self.fc2 = nn.Linear(256 ,64)
        self.out = nn.Linear(64, self.n_output)

    def forward(self, pro1_data, pro2_data):

        #get graph input for protein 1 
        pro1_x, pro1_edge_index, pro1_batch = pro1_data.x, pro1_data.edge_index, pro1_data.batch
        # get graph input for protein 2
        pro2_x, pro2_edge_index, pro2_batch = pro2_data.x, pro2_data.edge_index, pro2_data.batch

        x = self.pro1_conv1(pro1_x, pro1_edge_index)
        x = self.relu(x)
        
	# global pooling
        x = gep(x, pro1_batch)   

        # flatten
        x = self.relu(self.pro1_fc1(x))
        x = self.dropout(x)

        xt = self.pro2_conv1(pro2_x, pro2_edge_index)
        xt = self.relu(xt)

	# global pooling
        xt = gep(xt, pro2_batch)  

        # flatten
        xt = self.relu(self.pro2_fc1(xt))
        xt = self.dropout(xt)

	# Concatenation  
        xc = torch.cat((x, xt), 1)

        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        out = self.sigmoid(out)
        return out
        

net = GCNN()
print(net)

"""# GAT"""

class AttGNN(nn.Module):
    def __init__(self, n_output=1, num_features_pro= 1024, output_dim=128, dropout=0.2, heads = 1 ):
        super(AttGNN, self).__init__()

        print('AttGNN Loaded')

        self.hidden = 8
        self.heads = 1
        
        # for protein 1
        self.pro1_conv1 = GATConv(num_features_pro, self.hidden* 16, heads=self.heads, dropout=0.2)
        self.pro1_fc1 = nn.Linear(128, output_dim)


        # for protein 2
        self.pro2_conv1 = GATConv(num_features_pro, self.hidden*16, heads=self.heads, dropout=0.2)
        self.pro2_fc1 = nn.Linear(128, output_dim)

        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout)
        # combined layers
        self.fc1 = nn.Linear(2 * output_dim, 256)
        self.fc2 = nn.Linear(256, 64)
        self.out = nn.Linear(64, n_output)
        


    def forward(self, pro1_data, pro2_data):

        # get graph input for protein 1 
        pro1_x, pro1_edge_index, pro1_batch = pro1_data.x, pro1_data.edge_index, pro1_data.batch
        # get graph input for protein 2
        pro2_x, pro2_edge_index, pro2_batch = pro2_data.x, pro2_data.edge_index, pro2_data.batch
         
        
        x = self.pro1_conv1(pro1_x, pro1_edge_index)
        x = self.relu(x)
        
	# global pooling
        x = gep(x, pro1_batch)  
       
        # flatten
        x = self.relu(self.pro1_fc1(x))
        x = self.dropout(x)



        xt = self.pro2_conv1(pro2_x, pro2_edge_index)
        xt = self.relu(self.pro2_fc1(xt))
	
	# global pooling
        xt = gep(xt, pro2_batch)  

        # flatten
        xt = self.relu(xt)
        xt = self.dropout(xt)

	
	# Concatenation
        xc = torch.cat((x, xt), 1)

        # add some dense layers
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        out = self.sigmoid(out)
        return out

net_GAT = AttGNN()
print(net_GAT)



GCNN Loaded
GCNN(
  (pro1_conv1): GCNConv(1024, 1024)
  (pro1_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (pro2_conv1): GCNConv(1024, 1024)
  (pro2_fc1): Linear(in_features=1024, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
AttGNN Loaded
AttGNN(
  (pro1_conv1): GATConv(1024, 128, heads=1)
  (pro1_fc1): Linear(in_features=128, out_features=128, bias=True)
  (pro2_conv1): GATConv(1024, 128, heads=1)
  (pro2_fc1): Linear(in_features=128, out_features=128, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (sigmoid): Sigmoid()
  (dropout): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=256, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (out): 

In [57]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
from metrics import *

# utilities
def train(model, trainloader, optimizer, epoch):
    print(f'Training on {len(trainloader)} samples.....')
    model.train()
    loss_func = nn.MSELoss()
    predictions_tr = torch.Tensor()
    scheduler = MultiStepLR(optimizer, milestones=[1, 5], gamma=0.5)
    labels_tr = torch.Tensor()
    for count, (prot_1, prot_2, label) in enumerate(trainloader):
        optimizer.zero_grad()
        # output = model(prot_1, prot_2)
        # predictions_tr = torch.cat((predictions_tr, output.cpu()), 0)
        # labels_tr = torch.cat((labels_tr, label.view(-1, 1).cpu()), 0)
        # loss = loss_func(output, label.view(-1, 1).float())
        # loss.backward()
        # optimizer.step()
    # scheduler.step()
    # labels_tr = labels_tr.detach().numpy()
    # predictions_tr = predictions_tr.detach().numpy()
    # acc_tr = get_accuracy(labels_tr, predictions_tr, 0.5)
    # print(f'Epoch {epoch-1} / 30 [==============================] - train_loss : {loss} - train_accuracy : {acc_tr}')


def predict(model, loader):
    model.eval()
    predictions = torch.Tensor()
    labels = torch.Tensor()
    with torch.no_grad():
        for prot_1, prot_2, label in loader:
            output = model(prot_1, prot_2)
            predictions = torch.cat((predictions, output.cpu()), 0)
            labels = torch.cat((labels, label.view(-1, 1).cpu()), 0)
    labels = labels.numpy()
    predictions = predictions.numpy()
    return labels.flatten(), predictions.flatten()


# training
n_epochs_stop = 6
epochs_no_improve = 0
early_stop = False

model = GCNN()
num_epochs = 50
loss_func = nn.MSELoss()
min_loss = 100
best_accuracy = 0
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(optimizer)


print("")
print("")

# for epoch in range(num_epochs):
for epoch in range(1):
    train(model, trainloader, optimizer, epoch+1)

#     G, P = predict(model, testloader)
#     loss = get_mse(G, P)
#     accuracy = get_accuracy(G, P, 0.5)
#     print(f'Epoch {epoch} / {num_epochs} [==============================] - val_loss : {loss} - val_accuracy : {accuracy}')
#     if accuracy > best_accuracy:
#         best_accuracy = accuracy
#         best_acc_epoch = epoch
#         torch.save(model.state_dict(), "../human_features/GCN.pth")  # path to save the model
#         print("Model")
#     if loss < min_loss:
#         epochs_no_improve = 0
#         min_loss = loss
#         min_loss_epoch = epoch
#     elif loss > min_loss:
#         epochs_no_improve += 1
#     if epoch > 5 and epochs_no_improve == n_epochs_stop:
#         print('Early stopping!')
#         early_stop = True
#         break

# print(f'min_val_loss : {min_loss} for epoch {min_loss_epoch} ............... best_val_accuracy : {best_accuracy} for epoch {best_acc_epoch}')
# print("Model saved")


GCNN Loaded
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


Training on 4444 samples.....
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\laura\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\laura\AppData\Local\Temp\ipykernel_9020\475155841.py", line 62, in <module>
    train(model, trainloader, optimizer, epoch+1)
  File "C:\Users\laura\AppData\Local\Temp\ipykernel_9020\475155841.py", line 14, in train
    for count, (prot_1, prot_2, label) in enumerate(trainloader):
  File "c:\Users\laura\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 681, in __next__
    data = self._next_data()
  File "c:\Users\laura\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 721, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "c:\Users\laura\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "c:\Users\laura\anaconda3\lib\site-packages

In [2]:
import pandas as pd

# Assuming final_pairs is a NumPy array containing the data
df = pd.DataFrame(final_pairs)
df

,0,1,2,3,4,5,6
0,NP_663777,Q13114,1FLK,NP_001233,P26842,5TL5,1
1,NP_003630,Q9Y6K9,2JVX,NP_001073594,Q14790,1F9E,1
2,NP_003001,P45985,3ALN,NP_006292,Q12852,5CEN,1
3,NP_060901,Q9P104,1J0W,NP_000199,P06213,1GAG,1
4,NP_001020276,P48729,5FQD,NP_001895,P35222,1G3J,1
...,...,...,...,...,...,...,...
22212,NP_000240,P40692,3RBN,NP_000404,P14061,1A27,0
22213,P10619,P10619,1IVY,NP_005989,P49368,6NR8,0
22214,NP_000779,P27707,1P5Z,NP_005557,P00338,1I10,0
22215,NP_008828,P31274,2LP0,NP_434700,Q9H257,6E25,0


In [68]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [69]:
df

,0,1,2,3,4,5,6
0,NP_663777,Q13114,1FLK,NP_001233,P26842,5TL5,1
1,NP_003630,Q9Y6K9,2JVX,NP_001073594,Q14790,1F9E,1
2,NP_003001,P45985,3ALN,NP_006292,Q12852,5CEN,1
3,NP_060901,Q9P104,1J0W,NP_000199,P06213,1GAG,1
4,NP_001020276,P48729,5FQD,NP_001895,P35222,1G3J,1
...,...,...,...,...,...,...,...
22212,NP_000240,P40692,3RBN,NP_000404,P14061,1A27,0
22213,P10619,P10619,1IVY,NP_005989,P49368,6NR8,0
22214,NP_000779,P27707,1P5Z,NP_005557,P00338,1I10,0
22215,NP_008828,P31274,2LP0,NP_434700,Q9H257,6E25,0


In [82]:
# Step 1: Data Preprocessing

# Convert non-numeric columns to numeric types or exclude them
numeric_cols = df.select_dtypes(include=[np.number]).columns
df_numeric = df[numeric_cols]

protein_ids = pd.unique(df[[2, 5]].values.flatten())  # Extract protein IDs and get unique values
target_labels = df[6].values  # Extract interaction labels
features = df_numeric.iloc[:, 7:].values.astype(float)  # Extract numeric features
features


array([], shape=(22217, 0), dtype=float64)

In [83]:
# Step 2: Graph Construction
protein_dict = {protein_ids[i]: i for i in range(len(protein_ids))}  # Create a dictionary mapping protein IDs to indices

edge_index = []
for row in df[[2, 5]].itertuples(index=False):
    src = protein_dict[row[0]]
    tgt = protein_dict[row[1]]
    edge_index.append((src, tgt))

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_index

tensor([[   0,    2,    4,  ..., 3825, 3749,  878],
        [   1,    3,    5,  ..., 3162, 3777,  888]])

In [84]:
# Step 3: Feature Engineering
x = torch.tensor(features, dtype=torch.float)
x

tensor([], size=(22217, 0))

In [85]:
# Step 4: Model Architecture
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [86]:
df.columns[7:]

RangeIndex(start=7, stop=7, step=1)

In [87]:
# Step 5: Training
protein_columns = [2, 5]  # Assuming column indices 2 and 5 are the protein identifier columns
numeric_cols = df.columns.difference(protein_columns)

# Extract features from the numeric columns
features = df.loc[:, numeric_cols].values.astype(float)

x = torch.tensor(features, dtype=torch.float)
x

# #######################################


# model = GNNModel(input_dim=features.shape[1], hidden_dim=64, num_classes=1)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# data = Data(x=x, edge_index=edge_index, y=torch.tensor(target_labels, dtype=torch.float))
# data = train_test_split_edges(data)  # Perform train-test split for link prediction

# def train():
#     model.train()
#     optimizer.zero_grad()
#     out = model(data.x, data.train_pos_edge_index)
#     loss = F.binary_cross_entropy_with_logits(out.view(-1), data.train_pos_edge_label)
#     loss.backward()
#     optimizer.step()
#     return loss.item()

# def test():
#     model.eval()
#     out = model(data.x, data.test_neg_edge_index)
#     pred = (out.view(-1) > 0).float()
#     acc = (pred == data.test_neg_edge_label).sum().item() / data.test_neg_edge_label.size(0)
#     return acc

# for epoch in range(1, 101):
#     loss = train()
#     acc = test()
#     print(f'Epoch: {epoch}, Loss: {loss:.4f}, Accuracy: {acc:.4f}')

ValueError: could not convert string to float: 'NP_663777'